In [1]:
import  pandas as pd
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("titanic")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "hadoop")
    .config("spark.sql.catalog.demo.warehouse", "s3a://bronze/warehouse/")
    .config("spark.hadoop.fs.s3a.access.key", "teste")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/10/09 21:05:39 WARN Utils: Your hostname, DESKTOP-4LPUPP7 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/09 21:05:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/andre/miniforge3/envs/iceberg/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andre/.ivy2/cache
The jars for the packages stored in: /home/andre/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c1095477-9f4d-4b6a-9c08-cbb4ec34cc54;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 166ms :: artifacts dl 9ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	-------------------

In [2]:
%load_ext 00-prettytables

✅ Spark SQL Magic carregado! Use %sql ou %%sql para executar consultas Spark SQL.


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv")


In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


25/10/09 21:05:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
%%sql
select  1

1
1


In [6]:
%%sql
SELECT * FROM demo.db_teste_iceberg.teste

25/10/09 21:06:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


id,nome,valor
3,C,11.5
1,A,10.5
2,B,20.0
4,B,21.0


In [11]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS demo.ml_titanic_raw.titanic
""")

print("✅ Tabela criada com sucesso!")

✅ Tabela criada com sucesso!


In [12]:
spark_df = spark.createDataFrame(df)
spark_df.write.mode("overwrite").saveAsTable("demo.ml_titanic_raw.titanic")


In [ ]:
%%sql
select * from demo.ml_titanic_raw.titanic limit 100

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NaN,S
6,0,3,"Moran, Mr. James",male,nan,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master Gosta Leonard",male,2.0,3,1,349909,21.075,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [9]:
%%sql 

DROP TABLE IF EXISTS demo.ml_titanic_raw.titanic PURGE

++
||
++
++

In [10]:
%%sql
select sex, count(*) from demo.ml_titanic_raw.titanic group by sex

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `demo`.`ml_titanic_raw`.`titanic` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 26;
'Aggregate ['sex], ['sex, unresolvedalias(count(1), None)]
+- 'UnresolvedRelation [demo, ml_titanic_raw, titanic], [], false


In [8]:
%%sql 

SELECT * FROM demo.db_teste_iceberg.teste

id,nome,valor
3,C,11.5
1,A,10.5
4,B,21.0
2,B,20.0
